# DATA Extraction

In [1]:
import os
from collections import defaultdict
import sys
import time
import json
import psutil
import argparse
import operator
import pandas as pd
sys.path.append("/nykaa/api")
from pas.v2.utils import Utils

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
nykaadb = Utils.nykaaMysqlConnection()
cursor = nykaadb.cursor()

Query for Extracting data with limit

In [3]:
customer_orders_query_with_limit = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item sfoi INNER JOIN sales_flat_order sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND sfo.customer_id IS NOT NULL LIMIT 50000;"

Query for Extracting data without limit

In [4]:
customer_orders_query_without_limit = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item sfoi INNER JOIN sales_flat_order sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfo.customer_id IS NOT NULL;"
#customer_orders_query_without_limit = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item sfoi INNER JOIN sales_flat_order sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND sfo.customer_id IS NOT NULL;"

Query for Extracting data from archive table

In [5]:
customer_orders_archive_query = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item_archive sfoi INNER JOIN sales_flat_order_archive sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfo.customer_id IS NOT NULL;"
#customer_orders_archive_query = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item_archive sfoi INNER JOIN sales_flat_order_archive sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND sfo.customer_id IS NOT NULL;"

In [ ]:
def extract_data(query):
    cursor.execute(query)
    rows = []
    BATCH_SIZE = 10000

    while True:
        batch_empty = True
        for row in cursor.fetchmany(BATCH_SIZE):
            batch_empty = False
            rows.append(row)
        if batch_empty:
            break
            
    return rows

Dataframe for storing extracted data

In [6]:
df = pd.DataFrame()

Extracting data without limits

In [7]:
rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), customer_orders_query_without_limit, 10000)
df = df.append(rows)
len(rows), len(df)

(44461993, 44461993)

In [8]:
df.head()

,0,1,2,3,4,5
0,1867994,197355,19050,configurable,Nature's Essence Magic Henna Hair Color,2016-10-21 00:07:42
1,1867994,197355,19048,simple,Nature's Essence Magic Henna Hair Color Burgundy,2016-10-21 00:07:42
2,1867994,197355,12721,configurable,Aroma Magic Aromatic Skin Toner,2016-10-21 00:07:42
3,1867994,197355,12492,simple,Aroma Magic Aromatic Skin Toner,2016-10-21 00:07:42
4,1867994,197355,13417,configurable,Aroma Magic Sunblock Lotion,2016-10-21 00:07:42


Extracting data with limits

In [ ]:
rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), customer_orders_query_with_limit, 10000)
df = df.append(rows)
len(rows), len(df)

Extracting archive data

In [9]:
rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), customer_orders_archive_query, 10000)
df = df.append(rows)
len(rows), len(df)

(6990225, 51452218)

In [10]:
df.head()

,0,1,2,3,4,5
0,1867994,197355,19050,configurable,Nature's Essence Magic Henna Hair Color,2016-10-21 00:07:42
1,1867994,197355,19048,simple,Nature's Essence Magic Henna Hair Color Burgundy,2016-10-21 00:07:42
2,1867994,197355,12721,configurable,Aroma Magic Aromatic Skin Toner,2016-10-21 00:07:42
3,1867994,197355,12492,simple,Aroma Magic Aromatic Skin Toner,2016-10-21 00:07:42
4,1867994,197355,13417,configurable,Aroma Magic Sunblock Lotion,2016-10-21 00:07:42


In [11]:
df.columns = ['order_id', 'customer_id', 'product_id', 'product_type', 'product_name', 'created_at']

In [12]:
df.head()

,order_id,customer_id,product_id,product_type,product_name,created_at
0,1867994,197355,19050,configurable,Nature's Essence Magic Henna Hair Color,2016-10-21 00:07:42
1,1867994,197355,19048,simple,Nature's Essence Magic Henna Hair Color Burgundy,2016-10-21 00:07:42
2,1867994,197355,12721,configurable,Aroma Magic Aromatic Skin Toner,2016-10-21 00:07:42
3,1867994,197355,12492,simple,Aroma Magic Aromatic Skin Toner,2016-10-21 00:07:42
4,1867994,197355,13417,configurable,Aroma Magic Sunblock Lotion,2016-10-21 00:07:42


In [13]:
products_df = df.filter(['product_id', 'product_name'])
products_df.head()

,product_id,product_name
0,19050,Nature's Essence Magic Henna Hair Color
1,19048,Nature's Essence Magic Henna Hair Color Burgundy
2,12721,Aroma Magic Aromatic Skin Toner
3,12492,Aroma Magic Aromatic Skin Toner
4,13417,Aroma Magic Sunblock Lotion


In [14]:
product_id_2_name = dict(zip(products_df.product_id, products_df.product_name))
print(len(product_id_2_name))
del products_df

122333


In [15]:
order_date_df = df.filter(['order_id', 'created_at'])
order_date_df.head()

,order_id,created_at
0,1867994,2016-10-21 00:07:42
1,1867994,2016-10-21 00:07:42
2,1867994,2016-10-21 00:07:42
3,1867994,2016-10-21 00:07:42
4,1867994,2016-10-21 00:07:42


In [16]:
order_id_2_date = dict(zip(order_date_df.order_id, order_date_df.created_at))
print(len(order_id_2_date))
del order_date_df

9769375


# Parallelizing Dataframe operations

In [17]:
from multiprocessing import Pool
import numpy as np

def parallelize_dataframe(df, func, num_partitions, num_cores):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Preparing child to parent dict for products

In [18]:
query = "select child_id, parent_id from catalog_product_relation"

rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), query, 50000)
child_2_parent = {row[0]: row[1] for row in rows}

In [19]:
with open("child_product_2_parent.json", "r+") as f:
    child_2_parent.update({int(key): int(value) for key, value in json.load(f).items()})

In [20]:
def convert_2_parent_product(data):
    data['product_id'] = data.apply(lambda row: child_2_parent.get(row['product_id'], row['product_id']), axis=1)
    return data

In [21]:
df = parallelize_dataframe(df, convert_2_parent_product, 40, 16)

# Removing duplicate products from an order

In [22]:
df = df.filter(['customer_id', 'order_id', 'product_id'])

In [23]:
df['product_count'] = 1

In [24]:
df = df.groupby(['customer_id', 'order_id', 'product_id']).agg({'product_count': 'count'}).reset_index()

In [25]:
len(df)

33430418

In [26]:
df = df.drop(['product_count'], axis=1)

In [27]:
df.head()

,customer_id,order_id,product_id
0,20,3558386,28977
1,20,8018927,76415
2,20,8018927,216526
3,20,9255244,68705
4,20,9255244,255369


# Adding is_luxe field to data

First extracting out all the luxe products

In [ ]:
import sys
sys.path.append("/nykaa/api")
from pas.v2.utils import Utils

unique_product_ids = list(map(lambda x: int(x), list(df['product_id'].unique())))
es_query = {
    "query": {
        "bool": {
            "must": [
                {"term": {"is_luxe": True}},
                {"terms": {"product_id.keyword": unique_product_ids}}
            ]
        }
    },
    "_source": ["product_id"]
}

es_conn = Utils.esConn()
scroll_id = None
luxe_products = []
while True:
    if not scroll_id:
        response = es_conn.search(index='livecore', body=es_query, scroll='25m')
    else:
        response = es_conn.scroll(scroll_id=scroll_id, scroll='25m')
        
    if not response['hits']['hits']:
        break
      
    scroll_id = response['_scroll_id']
    luxe_products += [int(hit['_source']['product_id']) for hit in response['hits']['hits']]
    
luxe_products_dict = {int(product_id): True for product_id in luxe_products}

In [ ]:
luxe_products_dict[10839]

Now adding the field extracted

In [ ]:
df['is_luxe'] = df['product_id'].apply(lambda x: luxe_products_dict[int(x)] if luxe_products_dict.get(int(x)) else False)

In [ ]:
df.head()

In [ ]:
df[df['is_luxe'] == True].head()

# Writing to CSV File

In [28]:
df.to_csv('cab_6_july.csv', encoding='utf-8', index=False)

In [ ]:
count_query = "select count(*) from sales_flat_order_item_archive where mrp is NULL;"
cursor.execute(count_query)
row = cursor.fetchone()

In [ ]:
row

In [ ]:
nykaadb = Utils.nykaaMysqlConnection()
cursor = nykaadb.cursor()
count_query = "select count(*) from sales_flat_order_item where parent_item_id is null and order_id <> 0 and mrp=0;"
cursor.execute(count_query)
row = cursor.fetchone()
row